In [1]:
import pytest
import ipytest

ipytest.config(rewrite_asserts=True, magics=True)

current_notebook = 'pytest-101.ipynb'

__file__ = current_notebook

# Pytest 101

## Create test

Now you know how to do... it's so simple.

```python
def test_something():
    #blabla
    assert something
```

## Customize the assertion message

You could use the message feature of standard `assert`

```python 
assert condition, message
```

In [2]:
%%run_pytest[clean] -qq

def test_something():
    # given
    actual = 'A'
    expected = 'B'
    
    assert actual == expected, f'You have messed, {actual} is not {expected}'


F                                                                                                                [100%]
====================================================== FAILURES =======================================================
___________________________________________________ test_something ____________________________________________________

    def test_something():
        # given
        actual = 'A'
        expected = 'B'
    
>       assert actual == expected, f'You have messed, {actual} is not {expected}'
E       AssertionError: You have messed, A is not B
E       assert 'A' == 'B'
E         - A
E         + B

<ipython-input-2-ec3693ef7c6d>:6: AssertionError


There's some more customization available **per argument type** see [official documentation](http://doc.pytest.org/en/latest/assert.html#making-use-of-context-sensitive-comparisons)

## Check a exception  is raised

_More difficult._

Image the _Processing Under Test_ must raise an exception

In [3]:
def check_planet_name(name: str):
    if name not in 'Mercury Venus Earth Mars Jupiter Saturn Uranus Neptune'.split():
        raise ValueError(f'{name} is not a planet')
    

In [4]:
%%run_pytest[clean] -qq

def test_is_a_planet():
    check_planet_name('Mercury')
    
def test_is_a_planet():
    check_planet_name('foo')

F                                                                                                                [100%]
====================================================== FAILURES =======================================================
__________________________________________________ test_is_a_planet ___________________________________________________

    def test_is_a_planet():
>       check_planet_name('foo')

<ipython-input-4-3528dc69e803>:5: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

name = 'foo'

    def check_planet_name(name: str):
        if name not in 'Mercury Venus Earth Mars Jupiter Saturn Uranus Neptune'.split():
>           raise ValueError(f'{name} is not a planet')
E           ValueError: foo is not a planet

<ipython-input-3-903e962f3fdb>:3: ValueError


Pytest uses a context manager `pytest.raises`

Simple one: just check if exception is raised

In [5]:
%%run_pytest[clean] -qq

def test_is_a_planet():
    with pytest.raises(ValueError):
        check_planet_name('foo')

.                                                                                                                [100%]


Detailled one: get the raised exception and inspect it.

In [6]:
%%run_pytest[clean] -qq
        
def test_is_a_planet_with_exception_detail():
    with pytest.raises(ValueError) as exception :
        check_planet_name('foo')        
    assert str(exception.value) == 'foo is not a planet'


.                                                                                                                [100%]


Matches the exception's message.

In [7]:
%%run_pytest[clean] -qq

def test_is_a_planet_with_matcher():
    with pytest.raises(ValueError, match=r'foo is*' ):
        check_planet_name('foo')

.                                                                                                                [100%]


For more information, please take a look at [pytest Raises Official Documentation](https://docs.pytest.org/en/latest/reference.html#pytest-raises)

## Fixtures

`Fixture` is a very powerful feature of pytest.

It make easy 

* factorisation and composition of common processing
* identification of setup and teardown steps
* to distinguish heavy and expensive operation for integration tests from lighweight and cheap operation for unit tests.


## What's a fixture ?

A factorized processing

Imagine a `Drone` class, powered by several `Engine`s and managed by a `DriverUnit`

In [8]:
class Drone:
    
    def __init__(self, engines, drive_unit):
        self.engines = engines
        self.drive_unit = drive_unit
        
        
    def start(self):
        for engine in self.engines:
            engine.start()
            
    def is_started(self):
        return all(engine.started for engine in self.engines)

To unit test the `Drone`  **in isolation of others object**, we create mock `Engine` and `DriveUnit`

In [9]:
class MockEngine:
    def start(self):
        self.status = 'started'
        
    def is_started(self):
        return self.status == 'started'
    
class MockDriveUnit:
    pass

In [10]:
%%run_pytest[clean] -q

def test_start_drone():
    # given
    engines = [MockEngine() for _ in range(1,4)]
    drone= Drone(engines=engines,drive_unit=MockDriveUnit())
    
    # When
    drone.start()
    
    #then
    assert all(engine.status == 'started' for engine in engines)

.                                                                                                                [100%]
1 passed in 0.01s


It's a little bit heavy to repeat no ?

What is the fundamental law of any programer ?

<center>DRY</center>
<div class="alert alert-info">
    <center>
    <b>
        <span style="font-size:larger;">DON'T<br>REPEAT<br>YOURSELF</span>
     </b>
    </center>
</div>

<center>Never,</center>
<center>ever</center>

So we can factorize the set up.

In [11]:
def init_context():
    engines = [MockEngine() for _ in range(1,4)]
    drive_unit = MockDriveUnit()
    drone= Drone(engines=engines,drive_unit=drive_unit)
    return drone, engines, drive_unit

In [12]:
%%run_pytest[clean] -q

def test_start_drone():
    # given
    drone, engines, drive_unit = init_context()
    
    # When
    drone.start()
    
    #then
    assert all(engine.status == 'started' for engine in engines)

.                                                                                                                [100%]
1 passed in 0.01s


_Pytest_ proposes a fixture mechanism to standardise this kind of factorisation.

**Firstly**

* Annotate a function with `@pytest.fixture` mark
* Make the function create the right context and return object of interest

**Secondly**

* Use the ***name*** of the function as argument of the test.
* Use the returned values from fixture

uhm ?

Illustration

In [13]:
import pytest

@pytest.fixture
def init_context():
    engines = [MockEngine() for _ in range(1,4)]
    drive_unit = MockDriveUnit()
    drone= Drone(engines=engines,drive_unit=drive_unit)
    return drone, engines, drive_unit    

In [14]:
%%run_pytest[clean] -q

def test_start_drone(init_context):
    # given
    drone, engines, drive_unit = init_context
    
    # When
    drone.start()
    
    #then
    assert all(engine.status == 'started' for engine in engines)

.                                                                                                                [100%]
1 passed in 0.01s


_But it's the same, no ?_

Not really, because it's a declarative way of injecting dependency.  
For the sake of simplicity, we put the fixture near the test.

But the _fixture_ could be 
* shared among several tests 
* and defined in other locations

Pytest will collect and inject them according to the name of the parameter.

As user, we don't have to care **how** to inject, _Pytest_ does it for us.

<center>
<img src="images/pytest-fixture-injection.png">
</center>

## Fixture composition

What appends if the fixture needs another common function ?

That's simple: it use the same mechanism :)

`def <consummer_name>(<dependency_name>)`

In [15]:
%%run_pytest[clean] -qq

import pytest

@pytest.fixture
def init_engines():
    engines = [MockEngine() for _ in range(1,4)]
    return engines

@pytest.fixture
def init_context(init_engines):
    engines = init_engines
    drive_unit = MockDriveUnit()
    drone= Drone(engines=engines,drive_unit=drive_unit)
    return drone, engines, drive_unit   

def test_start_drone(init_context):
    # given
    drone, engines, drive_unit = init_context
    # When
    drone.start()
    #then
    assert all(engine.status == 'started' for engine in engines)
    

.                                                                                                                [100%]


The _consummer_ can ask for as many dependencies as it need.

`def <consummer_name>(<dependency_name_0>,<dependency_name_1>,...,<dependency_name_p>)`

In [16]:
%%run_pytest[clean] -qq

import pytest

@pytest.fixture
def init_drive_unit():
     return MockDriveUnit()

@pytest.fixture
def init_engines():
    engines = [MockEngine() for _ in range(1,4)]
    return engines

@pytest.fixture
def init_context(init_engines, init_drive_unit):
    drone= Drone(engines=init_engines,drive_unit=init_drive_unit)
    return drone, init_engines, init_drive_unit

def test_start_drone(init_context):
    # given
    drone, engines, drive_unit = init_context
    # When
    drone.start()
    #then
    assert all(engine.status == 'started' for engine in engines)

.                                                                                                                [100%]


## Fixture setup and teardown

Sometimes we need _setup_ and _teardown_ phases before and after **each** test.


Imagine we get a Computer adding numbers from a file.

In [17]:
from my.Computer import Computer

class FileBasedComputer(Computer):
  
    def __init__(self,file):
        super().__init__()
        self.file = file
    
    def add_all(self):
        with open(self.file,'r') as f:
            self.add_collection(list(f.readlines()))
            
    def add_collection(self, collection):
        for s in collection:
            self.add(float(s))

The unit tests

In [18]:
%%run_pytest[clean] -qq


numbers = list(range(1,5))

@pytest.fixture
def init_file():
    filename = 'test-temp.txt'
    with open(filename,'w') as file:
        file.write('\n'.join([str(i) for i in numbers]))
    return filename

def test_add_collection(init_file):
    # given
    file_based_computer = FileBasedComputer(init_file)
    # when
    file_based_computer.add_all()
    # then
    assert file_based_computer.total == 10


.                                                                                                                [100%]


It works, but maybe do we want to teardown and clean everything after a test is done.
        

In [19]:
%%run_pytest[clean] -qq
import os


numbers = list(range(1,5))

@pytest.fixture
def init_file():
    filename = 'test-temp.txt'
    with open(filename,'w') as file:
        file.write('\n'.join([str(i) for i in numbers]))
        
    yield filename
    
    os.remove(filename)
    

def test_add_collection(init_file):
    # given
    file_based_computer = FileBasedComputer(init_file)
    # when
    file_based_computer.add_all()
    # then
    assert file_based_computer.total == 10

.                                                                                                                [100%]


<center>
<img src="images/pytest-fixture-setup-teardown.png">
</center>

For ***Unit test*** it's not mandatory, but for ***Integration test*** or another heavy testing it's mandatory

<center>
<img src="images/pytest-heavy-setup-teardown.png">
</center>

Some setup has to be done for the whole test suite, as creating Database, starting an application, locking some resources.

But we can not create DB for each test; So we have to get different **scopes**

<center>
<img src="images/pytest-heavy-setup-teardown-different-scope.png">
</center>

Some operations are performed for the whole test suite, but others have to be renewed at each test.

They differ by **SCOPE**

## Fixture scoping

_Pytest_ use several scopes, the main are
* function,
* module,
* session

For more details see [Scope official documentation](https://docs.pytest.org/en/latest/fixture.html#scope-sharing-a-fixture-instance-across-tests-in-a-class-module-or-session)

### Function scope

* Fixture is ran at **each test**:
    * setup 
    * yield &rarr; run function `test_xxx(...)`
    * teardown 

<center>
<img src="images/pytest-scope-function.png">
</center>

## Module scope

*  Fixture is ran at **each module** (file) :
    * setup 
    * yield &rarr; run ***all*** functions `test_xxx(...)` in `test_nnn.py` file
    * teardown 

<center>
<img src="images/pytest-scope-module.png">
</center>

### Session scope

*  Fixture is ran **once** for the whole pytest session:
    * setup 
    * yield &rarr; run ***all tests*** found by pytest
    * teardown 
    
It's very useful for setting up and teardown of heavy and expensive resources consumming tests.


<center>
<img src="images/pytest-scope-session.png">
</center>

## Fixture conftest

TBD